In [1]:
# %%
import keras, tensorflow as tf


2024-07-26 09:07:17.771502: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 09:07:17.771580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 09:07:17.771614: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 09:07:17.782440: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# %%
model = keras.models.load_model("keyword_spotting/trained_models/kws_ref_model")


2024-07-26 09:10:29.519386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-26 09:10:29.525943: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-26 09:10:29.526010: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-26 09:10:29.529277: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-26 09:10:29.529339: I tensorflow/compile

In [4]:
# %%
model.summary()
# keras.utils.plot_model(model) need more packages for visualization, used netron instead
for layer in model.layers:
    print(layer.get_config()['name'], layer.get_config()['dtype'])


Model: "functional_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 49, 10, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 25, 5, 64)         2624      
                                                                 
 batch_normalization (Batch  (None, 25, 5, 64)         256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 25, 5, 64)         0         
                                                                 
 dropout (Dropout)           (None, 25, 5, 64)         0         
                                                                 
 depthwise_conv2d (Depthwis  (None, 25, 5, 64)         640       
 eConv2D)                                             

In [5]:
# %%
# Initialize the interpreter
model_path = 'keyword_spotting/trained_models/kws_ref_model.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

tensor_details = interpreter.get_tensor_details()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

for detail in tensor_details:
    print(detail['name'])
    print('\t', detail['shape'])
    print('\t', detail['dtype'])
    # print('\t', detail['quantization_parameters'])

print("Input details:")
print(input_details)
print("Output details")
print(output_details)


input_1
	 [ 1 49 10  1]
	 <class 'numpy.int8'>
functional_1/dense/BiasAdd/ReadVariableOp/resource
	 [12]
	 <class 'numpy.int32'>
functional_1/flatten/Const
	 [2]
	 <class 'numpy.int32'>
functional_1/activation/Relu;functional_1/batch_normalization/FusedBatchNormV3;functional_1/conv2d/BiasAdd/ReadVariableOp/resource;functional_1/conv2d/BiasAdd;functional_1/conv2d_4/Conv2D;functional_1/conv2d/Conv2D
	 [64]
	 <class 'numpy.int32'>
functional_1/activation_1/Relu;functional_1/batch_normalization_1/FusedBatchNormV3;functional_1/depthwise_conv2d/depthwise;functional_1/depthwise_conv2d/BiasAdd;functional_1/conv2d_4/Conv2D;functional_1/depthwise_conv2d/BiasAdd/ReadVariableOp/resource
	 [64]
	 <class 'numpy.int32'>
functional_1/batch_normalization_1/FusedBatchNormV3;functional_1/depthwise_conv2d/depthwise;functional_1/depthwise_conv2d/BiasAdd;functional_1/conv2d_4/Conv2D;functional_1/depthwise_conv2d/BiasAdd/ReadVariableOp/resource
	 [ 1  3  3 64]
	 <class 'numpy.int8'>
functional_1/activation_2

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
